## Importing Libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools

cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

In [2]:
device

device(type='cpu')

## Data Preprocessing

In [3]:
lines_filepath = os.path.join("cornell movie-dialogs corpus", "movie_lines.txt")
conv_filepath = os.path.join("cornell movie-dialogs corpus", "movie_conversations.txt")

In [4]:
#Visualize Lines
with open(lines_filepath, 'r') as file:
    lines = file.readlines()
for line in lines[:8]:
    print(line.strip())

L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!
L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!
L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.
L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?
L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.
L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow
L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.
L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No


In [5]:
#Categorizing
line_fields = ["lineID","characterID", "movieID", "character", "text"]
lines = {}
with open(lines_filepath, 'r', encoding ='iso-8859-1') as f:
    for line in f:
        values = line.split(" +++$+++ ")
        lineObj = {}
        for i, field in enumerate(line_fields):
            lineObj[field] = values[i]
        lines[lineObj['lineID']] = lineObj
    

In [6]:
#Categorizing Conversation
conv_fields = ["character1ID","character2ID", "movieID", "utteranceID"]
conversation = []
with open(conv_filepath, 'r', encoding ='iso-8859-1') as f:
     for line in f:
        values = line.split(" +++$+++ ")
        convObj = {}
        for i, field in enumerate(conv_fields):
            convObj[field] =  values[i]
        lineIDs = eval(convObj["utteranceID"])
        convObj["lines"] = []
        for lineID in lineIDs:
            convObj["lines"].append(lines[lineID])
        conversation.append(convObj)

In [7]:
#Pairs of sentences
qa_pairs = []
for conv in conversation:
    for i in range(len(conv["lines"])-1):
        inputline  = conv["lines"][i]["text"].strip()
        targetline = conv["lines"][i+1]["text"].strip()
        if inputline and targetline:
            qa_pairs.append([inputline, targetline])
        
    

In [8]:
#Writing to a file

datafile = os.path.join("cornell movie-dialogs corpus","formatted_movie_lines.txt")
delimiter = "\t"
print("\nWriting into output file..")
with open(datafile , 'w', encoding = 'utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter =  delimiter)
    for pair in qa_pairs:        
        writer.writerow(pair)
print("\nDone")


Writing into output file..

Done


In [9]:
datafile = os.path.join("cornell movie-dialogs corpus","formatted_movie_lines.txt")
with open(datafile ,'rb') as file:
    lines = file.readlines()
for line in lines[:8]:
    print(line)

b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\r\r\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\r\r\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\r\r\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\r\r\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\r\r\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\r\r\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\tSeems like she could get a date easy enough...\r\r\n"
b'Why?\tU

In [10]:
PAD_token = 0 #For padding short sentences
SOS_token = 1 #For start of a sentence
EOS_token = 2 #For the end of a sentence

class Vocabulary:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 #Count EOS SOS PAD
        
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addword(word)
    
    def addword(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words+=1
        else:
            self.word2count[word] += 1

            
    
    def trim(self, min_count):
        keep_words = []
        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)
        print('keep_words {} / {} = {:.4f}'.format(len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)))
        
        #Reinitialize 
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 #Count EOS SOS PAD
        
        for word in keep_words:
            self.addword(word)

In [11]:
#Unicode to ASCII
def unicodeToAscii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c)!='Mn')
    

In [12]:
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1",s)
    s = re.sub(r"[^a-zA-z.!?]+", r" " ,s)
    s = re.sub(r"\s", r" ", s)
    return s

In [13]:
normalizeString("aa12 ?11")

'aa ? '

In [14]:
datafile = os.path.join("cornell movie-dialogs corpus","formatted_movie_lines.txt")
print("Reading into the file..")
lines = open(datafile, encoding = 'utf-8').read().strip().split('\n')
pairs = [[normalizeString(s) for s in pair.split('\t')] for pair in lines]
print('Done Reading')
voc = Vocabulary("cornell movie-dialogs corpus")

Reading into the file..
Done Reading


In [15]:
#Filter
MAXLEN = 10
def filterPair(p):
    return len(p[0].split()) < MAXLEN and len(p[1].split())  < MAXLEN

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [16]:
pairs  = [pair for pair in pairs if len(pair)>1]
print(" There are {} pairs/conversation before filering".format(len(pairs)))
pairs = filterPairs(pairs)
print(" After filtering, there are {} pairs/conversations".format(len(pairs)))

 There are 221282 pairs/conversation before filering
 After filtering, there are 64266 pairs/conversations


In [17]:
for pair in pairs:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])
print("Counted words:", voc.num_words)
for pair in pairs[:10]:
    print(pair)

Counted words: 18077
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', ' the real you . ']


In [18]:
MINLEN = 3

def trimwords(voc, pairs, MINLEN):
    voc.trim(MINLEN)
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs)/len(pairs)))
    return keep_pairs       
pairs = trimwords(voc, pairs, MINLEN)

keep_words 7837 / 18074 = 0.4336
Trimmed from 64266 pairs to 53115, 0.8265 of total


In [19]:
def indexFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

In [20]:
indexFromSentence(voc, pairs[1][0])

[7, 8, 9, 10, 4, 11, 12, 13, 2]

In [21]:
inp = []
out = []
for pair in pairs[:10]:
    inp.append(pair[0])
    out.append(pair[1])
print(inp)
print(len(inp))
indexes = [indexFromSentence(voc, sentence) for sentence in inp]
indexes

['there .', 'you have my word . as a gentleman', 'hi .', 'have fun tonight ?', 'well no . . .', 'then that s all you had to say .', 'but', 'do you listen to this crap ?', 'what good stuff ?', 'wow']
10


[[3, 4, 2],
 [7, 8, 9, 10, 4, 11, 12, 13, 2],
 [16, 4, 2],
 [8, 31, 22, 6, 2],
 [33, 34, 4, 4, 4, 2],
 [35, 36, 37, 38, 7, 39, 40, 41, 4, 2],
 [42, 2],
 [47, 7, 48, 40, 45, 49, 6, 2],
 [50, 51, 52, 6, 2],
 [59, 2]]

In [22]:
def zeroPadding( l, fillvalue = 0):
    return list(itertools.zip_longest(*l, fillvalue = fillvalue))

In [23]:
length = [len(ind) for ind in indexes]
max(length)

10

In [24]:
#Test the function
test_result  = zeroPadding(indexes)
print(len(test_result))
test_result

10


[(3, 7, 16, 8, 33, 35, 42, 47, 50, 59),
 (4, 8, 4, 31, 34, 36, 2, 7, 51, 2),
 (2, 9, 2, 22, 4, 37, 0, 48, 52, 0),
 (0, 10, 0, 6, 4, 38, 0, 40, 6, 0),
 (0, 4, 0, 2, 4, 7, 0, 45, 2, 0),
 (0, 11, 0, 0, 2, 39, 0, 49, 0, 0),
 (0, 12, 0, 0, 0, 40, 0, 6, 0, 0),
 (0, 13, 0, 0, 0, 41, 0, 2, 0, 0),
 (0, 2, 0, 0, 0, 4, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 2, 0, 0, 0, 0)]

In [25]:
def binaryMatrix( l , value = 0):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

In [26]:
binary_result =  binaryMatrix(test_result)
binary_result

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 1, 0],
 [0, 1, 0, 1, 1, 1, 0, 1, 1, 0],
 [0, 1, 0, 1, 1, 1, 0, 1, 1, 0],
 [0, 1, 0, 0, 1, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]]

In [27]:
def inputVar( l, voc):
    indexes_batch = [indexFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(index) for index in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

In [28]:
def outputVar(l,voc):
    indexes_batch = [indexFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(index) for index in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask  = binaryMatrix(padList)
    mask  = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

In [29]:
def trainDataBatch(voc, pair_batch):
    pair_batch.sort(key = lambda x: len(x[0].split(' ')), reverse = True)
    input_batch, output_batch = [],[]
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, mask_max_leng = outputVar(output_batch, voc)
    return inp, lengths, output, mask, mask_max_leng

In [30]:
batch_size = 5
batches = trainDataBatch(voc , [random.choice(pairs) for _ in range(batch_size)] )
input_var , length, target_var, mask , max_target = batches
print("Input_Var")
print(input_var)
print("Target_Var")
print(target_var)

print("mask")
print(mask)
print("max_target")
print(max_target)

Input_Var
tensor([[  95,    7,   50,    5,    3],
        [  25,   75,   47,  212,   37],
        [ 402,   25,    7, 2158,  320],
        [ 588,  198,  119,  145,  327],
        [ 123,  118,  145, 1359,  350],
        [ 360,   24,   84,  200,    4],
        [   7,   36,    6,    6,    2],
        [ 197,    6,    2,    2,    0],
        [   6,    2,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
Target_Var
tensor([[  63, 1801,   25, 3293,   50],
        [   4,    4,    4, 3294,    6],
        [   2,  101,   25,    4,    2],
        [   0,   81,  198,    2,    0],
        [   0,  102,  118,    0,    0],
        [   0,   53,   24,    0,    0],
        [   0,    2,    4,    0,    0],
        [   0,    0,    2,    0,    0]])
mask
tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [0, 1, 1, 1, 0],
        [0, 1, 1, 0, 0],
        [0, 1, 1, 0, 0],
        [0, 1, 1, 0, 0],
        [0, 0, 1, 0, 0]], dtype=torch.uint8)
max_target
8


In [36]:
#Encoder
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers = 1, dropout =0):
        super(EncoderRNN,self).__init__()
        self.n_layers = n_layers
        self.embedding = embedding
        self.hidden_size = hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size , n_layers , dropout=( 0 if n_layers == 1 else dropout), bidirectional = True)
        
    def forward(self, input_seq, input_lengths,  hidden = None):
        embedded = self.embedding(input_seq)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        outputs, hidden = self.gru(packed, hidden)
        outputs, _ = torch.nn.utils.rnn.pack_padded_sequence(embedded, outputs)
        outputs = outputs[:,:, self_hidden_size]+ outputs[:,:, self_hidden_size:]
        return outputs,hidden

In [ ]:
#Attention Model
class Attn(torch.nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        self.hidden_size = hidden_size
        
    def dot_score(self, hidden, encoder_outputs):
        return torch.sum(hidden*encoder_output, dim = 2)
    
    def forward(self, hidden, encoder_outputs):
        attn_energies = self.dot_score(hidden, encoder_outputs)
        attn_energies = attn_energies.t()
        return F.softmax(attn_energies , din = 1).unsqeeze(1)